In [37]:
# Make sure you have added the dev version of SSM julia to the local project env using "dev /path/to/ssm_julia" in the package manager REPL
# Also add the Revise package to the local project env using "add Revise" in the package manager REPL

using Pkg
Pkg.activate(".")

using Random
using Distributions

using Revise
using SSM

  Activating project at `c:\Users\brook\Desktop\Carson\ssm_julia\notebooks\gaussian_regression`


In [38]:
function matrix(array)
    return reshape(array,1,:)
end

matrix (generic function with 1 method)

In [39]:
covariance = [0.1 0;
            0 0.1]
coefficients = [3 3;
                1 0.5;
                0.5 1]

model = GaussianRegression(coefficients, covariance, num_features=2, num_targets=2, include_intercept=true)

n = 1000
X = randn(n, 2)


y = SSM.predict(model, X)


# add some multidimensional gaussian noise with the covariance defined above 
y += rand(MvNormal(zeros(2), covariance), n)'

println("Data generated")

Data generated


In [40]:
covariance = [0.1 0;
            0 0.1]
coefficients = [3 3;
                .5 0.5;
                0.5 .5]

poor_model = GaussianRegression(coefficients, covariance, num_features=2, num_targets=2, include_intercept=true)

println("Log likelihood of the true model: ", SSM.surrogate_loglikelihood(model, X, y))
println("Log likelihood of the poor model: ", SSM.surrogate_loglikelihood(poor_model, X, y))

Log likelihood of the true model: -100.91084082247853
Log likelihood of the poor model: -349.27859240898596


In [41]:
#  Test the gradient function

G = zeros(3, 2)

surrogate_loglikelihood_gradient!(
        G, 
        GaussianRegression(
            poor_model.β, 
            poor_model.Σ, 
            num_features=model.num_features, 
            num_targets=model.num_targets, 
            include_intercept=model.include_intercept), 
        X, y)
println("Gradient of the log likelihood: ", G)

Gradient of the log likelihood: [19.40270167739677 0.9068681195086405; 523.7226401893729 -2.5668111444706274; 10.090092897707985 482.49598316786756]


In [42]:
# fit the poor model, then print the new log likelihood
SSM.fit!(poor_model, X, y)
println("New log likelihood of the poor model: ", SSM.surrogate_loglikelihood(poor_model, X, y))

made grad negative
[3.0140712096333258 3.0057037335642827; 0.9966770160867894 0.5015664410098529; 0.5148059784431198 0.9906457596588751]
New log likelihood of the poor model: -100.63900674064317


In [43]:
# pring the poor model parameters
println("Poor model coefficients: ", poor_model.β)
println("Poor model covariance: ", poor_model.Σ)

Poor model coefficients: [3.0140712096333258 3.0057037335642827; 0.9966770160867894 0.5015664410098529; 0.5148059784431198 0.9906457596588751]
Poor model covariance: [0.09857594368009825 0.00030928956158332045; 0.00030928956158332045 0.10270206980118812]
